<a href="https://colab.research.google.com/github/vagueau/Projet_python/blob/main/projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyarrow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

import pyarrow.parquet as pq

# Chemin du fichier Parquet
chemin_fichier = "/content/yellow_tripdata_2023-03.parquet"

# Lecture du fichier Parquet
table = pq.read_table(chemin_fichier)

# Conversion de la table en DataFrame pandas
dataframe = table.to_pandas()

# Affichage des données
print(dataframe)



FileNotFoundError: ignored

In [ ]:
dataframe.head()
dataframe.describe()



,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
count,3.403766e+06,3.316147e+06,3.403766e+06,3.316147e+06,3.403766e+06,3.403766e+06,3.403766e+06,3.403766e+06,3.403766e+06,3.403766e+06,3.403766e+06,3.403766e+06,3.403766e+06,3.403766e+06,3.316147e+06,3.316147e+06
mean,1.725320e+00,1.351417e+00,3.903871e+00,1.623961e+00,1.654540e+02,1.641003e+02,1.187310e+00,1.890845e+01,1.629128e+00,4.874393e-01,3.495237e+00,5.670059e-01,9.810640e-01,2.780343e+01,2.275176e+00,1.040236e-01
std,4.588677e-01,8.861919e-01,1.916866e+02,7.358691e+00,6.373958e+01,6.967282e+01,5.334025e-01,1.825637e+01,1.842778e+00,1.020516e-01,3.996473e+00,2.121658e+00,1.892314e-01,2.286614e+01,7.745563e-01,3.503260e-01
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-9.599000e+02,-7.500000e+00,-5.000000e-01,-8.000000e+01,-7.330000e+01,-1.000000e+00,-9.829500e+02,-2.500000e+00,-1.250000e+00
25%,1.000000e+00,1.000000e+00,1.050000e+00,1.000000e+00,1.320000e+02,1.140000e+02,1.000000e+00,9.300000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,1.000000e+00,1.570000e+01,2.500000e+00,0.000000e+00
50%,2.000000e+00,1.000000e+00,1.790000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,1.350000e+01,1.000000e+00,5.000000e-01,2.800000e+00,0.000000e+00,1.000000e+00,2.060000e+01,2.500000e+00,0.000000e+00
75%,2.000000e+00,1.000000e+00,3.380000e+00,1.000000e+00,2.330000e+02,2.340000e+02,1.000000e+00,2.120000e+01,2.500000e+00,5.000000e-01,4.340000e+00,0.000000e+00,1.000000e+00,2.976000e+01,2.500000e+00,0.000000e+00
max,6.000000e+00,9.000000e+00,2.169870e+05,9.900000e+01,2.650000e+02,2.650000e+02,5.000000e+00,2.100000e+03,1.375000e+01,4.000000e+00,9.843000e+02,1.770000e+02,1.000000e+00,2.100000e+03,2.500000e+00,1.250000e+00


In [ ]:
dataframe.dtypes

In [ ]:
dataframe.isnull().sum()

VendorID                     0
tpep_pickup_datetime         0
tpep_dropoff_datetime        0
passenger_count          87619
trip_distance                0
RatecodeID               87619
store_and_fwd_flag       87619
PULocationID                 0
DOLocationID                 0
payment_type                 0
fare_amount                  0
extra                        0
mta_tax                      0
tip_amount                   0
tolls_amount                 0
improvement_surcharge        0
total_amount                 0
congestion_surcharge     87619
Airport_fee              87619
dtype: int64